# Encontrando Conectores Chave em uma Rede Social

Este notebook explora conceitos básicos de análise de redes sociais, focando na identificação de "conectores chave" dentro de um grupo de usuários.

Vamos começar definindo a estrutura de dados para nossos usuários e suas amizades.

In [8]:
# Lista de usuários, onde cada usuário é um dicionário com 'id' e 'name'.
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

# Pares de amizade representando as conexões entre os usuários.
friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                    (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

# Inicializa o dicionário de amizades, onde a chave é o ID do usuário e o valor é uma lista de IDs de amigos.
# Cada usuário começa com uma lista vazia de amigos.
friendships = {user["id"]: [] for user in users}

# Preenche o dicionário de amizades com base nos pares de amizade.
# Para cada par (i, j), adiciona j como amigo de i e i como amigo de j (amizade bidirecional).
for i, j in friendship_pairs:
    friendships[i].append(j)  # Adiciona j como amigo do usuário i
    friendships[j].append(i)  # Adiciona i como amigo do usuário j

# Função para calcular o número de amigos de um usuário.
def number_of_friends(user):
    """Retorna o número de amigos que o usuário possui."""
    user_id = user["id"]
    friend_ids = friendships[user_id]
    return len(friend_ids)

# Calcula o total de conexões na rede somando o número de amigos de cada usuário.
# Nota: Cada conexão é contada duas vezes (uma para cada amigo no par), então o total de conexões
# é o dobro do número real de amizades.
total_connections = sum(number_of_friends(user) for user in users)

# Calcula o número total de usuários.
num_users = len(users)

# Calcula a média de conexões por usuário.
avg_connections = total_connections / num_users

# Cria uma lista de tuplas (user_id, number_of_friends) para todos os usuários.
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]

# Classifica a lista de usuários pelo número de amigos em ordem decrescente.
# Isso ajuda a identificar os "conectores chave" (usuários com mais amigos).
num_friends_by_id.sort(key=lambda id_and_friends: id_and_friends[1], reverse=True)

# Exemplo de impressão das conexões para alguns usuários (descomente para usar):
print(f"Amigos do usuário 0: {friendships[0]}") # Exemplo: [1, 2]
print(f"Amigos do usuário 1: {friendships[1]}") # Exemplo: [0, 2, 3]
print(f"Amigos do usuário 2: {friendships[2]}") # Exemplo: [0, 1, 3]

# Função (versão "ruim") para encontrar amigos de amigos (FOAF - Friend Of A Friend).
# Esta implementação pode retornar o próprio usuário ou amigos diretos como "amigos de amigos".
def foaf_ids_bad(user):
    """
    Retorna uma lista de IDs de amigos de amigos (FOAF).
    Esta versão inclui amigos diretos e o próprio usuário se houver um loop na rede.
    """
    return [foaf_id
            for friend_id in friendships[user["id"]]
            for foaf_id in friendships[friend_id]]

# Exemplos de uso da função foaf_ids_bad (descomente para usar):
print(f"-----------------------------------")
print(f"Amigos de amigos de Hero (id 0): {foaf_ids_bad(users[0])}")
print(f"Amigos de amigos de Dunn (id 1): {foaf_ids_bad(users[1])}")

Amigos do usuário 0: [1, 2]
Amigos do usuário 1: [0, 2, 3]
Amigos do usuário 2: [0, 1, 3]
-----------------------------------
Amigos de amigos de Hero (id 0): [0, 2, 3, 0, 1, 3]
Amigos de amigos de Dunn (id 1): [1, 2, 0, 1, 3, 1, 2, 4]


## Estrutura de Dados: Usuários e Amizades

Definimos uma lista de dicionários para representar nossos `users`, onde cada dicionário contém um `id` único e um `name`.

Em seguida, temos `friendship_pairs`, uma lista de tuplas que denota as amizades diretas entre os usuários, usando seus IDs.

Para facilitar a consulta, transformaremos `friendship_pairs` em um dicionário `friendships`, onde cada chave será o ID de um usuário e o valor será uma lista dos IDs de seus amigos.

## Calculando Conexões

Agora que temos nossa estrutura de amizades, podemos realizar algumas análises básicas:

- **`number_of_friends(user)`**: Uma função simples para contar quantos amigos um usuário tem.
- **`total_connections`**: A soma de todas as conexões na rede. Cada amizade é contada duas vezes (uma para cada pessoa envolvida no par).
- **`avg_connections`**: A média de conexões por usuário, oferecendo uma visão geral da densidade da rede.


## Amigos de Amigos (Friend Of A Friend - FOAF)

Um conceito importante em análise de redes é o de "amigos de amigos". A função `foaf_ids_bad` demonstra uma forma simples de encontrar amigos de amigos, embora esta versão possa incluir amigos diretos ou o próprio usuário em certos cenários. Em uma análise mais robusta, precisaríamos de uma lógica para evitar esses casos indesejados.


Como essas informações sobre quem é amigo de quem parecem
 ser interessantes, vamos gerar uma contagem de amigos em
 comum, porém excluindo as pessoas que o usuário já conhece:


In [9]:
from collections import Counter # Carrega a classe Counter para contagem de elementos

def friends_of_friends(user):
    """
    Encontra amigos de amigos (FOAF) de um usuário, excluindo o próprio usuário
    e seus amigos diretos.
    Retorna um objeto Counter com os IDs dos amigos de amigos e a frequência com que aparecem.
    """
    user_id = user["id"]
    return Counter(
        foaf_id
        for friend_id in friendships[user_id]      # Para cada amigo meu,
        for foaf_id in friendships[friend_id]      # encontre os amigos deles
        if foaf_id != user_id                      # que não sejam eu
        and foaf_id not in friendships[user_id]    # e não sejam meus amigos diretos.
    )

# Exemplo de uso da função friends_of_friends:
# Este exemplo mostra os amigos de amigos do usuário com id 3 (Chi).
# O resultado esperado é Counter({0: 2, 5: 1}), significando que o usuário 0 é amigo de amigos de Chi duas vezes, e o usuário 5 uma vez.
print(f"Amigos de amigos de Chi (id 3): {friends_of_friends(users[3])}")

Amigos de amigos de Chi (id 3): Counter({0: 2, 5: 1})


In [10]:
from collections import defaultdict, Counter

# Lista de interesses, onde cada tupla é (user_id, "interesse").
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

# Função para encontrar os IDs dos usuários que possuem um interesse específico.
def data_scientists_who_like(target_interest):
    """Encontra os IDs dos usuários com o mesmo interesse."""
    return [user_id
            for user_id, user_interest in interests
            if user_interest == target_interest]

# Exemplo de uso da função data_scientists_who_like:
print(f"Usuários que gostam de 'Python': {data_scientists_who_like('Python')}")
print(f"Usuários que gostam de 'Big Data': {data_scientists_who_like('Big Data')}")
print("-" * 40)

# Mapeia interesses para listas de IDs de usuários.
# As chaves são interesses, os valores são listas de user_ids com o interesse em questão.
user_ids_by_interest = defaultdict(list)
for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)

# Mapeia IDs de usuários para listas de seus interesses.
# As chaves são user_ids, os valores são listas de interesses do user_id em questão.
interests_by_user_id = defaultdict(list)
for user_id, interest in interests:
    interests_by_user_id[user_id].append(interest)

# Função para encontrar interesses em comum com outros usuários.
def most_common_interests_with(user):
    """
    Encontra os IDs de usuários que compartilham interesses com o usuário dado.
    Retorna um objeto Counter com os IDs dos usuários e o número de interesses em comum.
    """
    return Counter(
        interested_user_id
        for interest in interests_by_user_id[user["id"]]     # Para cada interesse do usuário,
        for interested_user_id in user_ids_by_interest[interest] # encontre outros usuários com esse interesse
        if interested_user_id != user["id"]                  # excluindo o próprio usuário.
    )

# Exemplos de uso da função most_common_interests_with:
# Isso nos mostra quem tem mais interesses em comum com Hero (id 0).
print(f"Interesses em comum com Hero (id 0): {most_common_interests_with(users[0])}")
# Isso nos mostra quem tem mais interesses em comum com Dunn (id 1).
print(f"Interesses em comum com Dunn (id 1): {most_common_interests_with(users[1])}")

Usuários que gostam de 'Python': [2, 3, 5]
Usuários que gostam de 'Big Data': [0, 8, 9]
----------------------------------------
Interesses em comum com Hero (id 0): Counter({9: 3, 1: 2, 8: 1, 5: 1})
Interesses em comum com Dunn (id 1): Counter({0: 2})
